# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [78]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key
gmaps.configure(api_key=g_key)



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [79]:
csv_path = "../WeatherPy/CitiesWeather.csv"
cityweather = pd.read_csv(csv_path, encoding="utf -8")
cityweather

,City,Country,Latitude,Longitude,Date,Temperature,Humidity,Cloudiness,Wind Speed
0,hobart,AU,-42.8794,147.3294,2021-01-14 12:05:10,13.42,94.0,75.0,0.51
1,albany,US,42.6001,-73.9662,2021-01-14 12:05:41,-0.62,73.0,99.0,0.95
2,maniitsoq,GL,65.4167,-52.9000,2021-01-14 12:05:42,-7.75,93.0,92.0,3.12
3,sungairaya,ID,0.7000,108.9000,2021-01-14 12:05:42,24.60,91.0,100.0,2.83
4,laramie,US,41.3114,-105.5911,2021-01-14 12:05:43,-8.79,57.0,1.0,7.20
...,...,...,...,...,...,...,...,...,...
544,levin,NZ,-40.6333,175.2750,2021-01-14 12:11:51,15.53,94.0,6.0,0.45
545,shirokiy,RU,49.7580,129.5230,2021-01-14 12:11:52,-28.26,86.0,0.0,1.29
546,bilibino,RU,68.0546,166.4372,2021-01-14 12:11:53,-29.81,84.0,88.0,0.39
547,kaduqli,SD,11.0167,29.7167,2021-01-14 12:11:53,35.43,9.0,9.0,2.83


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [80]:
# Configure gmaps with API key - I don't think I need to do this
#gmaps.configure(g_key=gkey)

In [81]:
print(cityweather["Latitude"].dtype)
print(cityweather["Longitude"].dtype)
print(cityweather["Humidity"].dtype)

float64
float64
float64


In [82]:
# Store 'Lat' and 'Lng' into  locations 
locations = cityweather[["Latitude", "Longitude"]]

# Convert Poverty Rate to float and store
# HINT: be sure to handle NaN values
#census_data_complete = census_data_complete.dropna()
humidity = cityweather["Humidity"]

print(len(locations))
print(len(humidity))

549
549


In [83]:
#locations = earthquake_df[['latitude', 'longitude']]
#weights = earthquake_df['magnitude']
#fig = gmaps.figure()
#fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
#fig

In [84]:
#import gmaps
#gmaps.configure(api_key=g_key)
#fig = gmaps.figure()
#locations = [(46.1, 5.2), (46.2, 5.3), (46.3, 5.4)]
#fig.add_layer(gmaps.heatmap_layer(locations))
#fig

In [85]:
# Create a Humidity Heatmap layer
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
#fig = gmaps.figure(center=(46.0, -5.0), zoom_level=8)

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 4)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [86]:
cityweather.columns

Index(['City', 'Country', 'Latitude', 'Longitude', 'Date', 'Temperature',
       'Humidity', 'Cloudiness', 'Wind Speed'],
      dtype='object')

In [87]:
cityweather["Wind Speed"].max()

13.21

In [88]:
#A max temperature lower than 30 degrees but higher than 20 = celsius.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is id

holiday = cityweather.loc[(cityweather["Temperature"]>20) & (cityweather["Temperature"]<30), :]

#cond16 = (df['daychange']<8) & (df['daychange']>2
holiday = holiday.loc[cityweather["Wind Speed"] < 10, :]
holiday = holiday.loc[cityweather["Cloudiness"] == 0, :]
holiday.index=range(0,len(holiday))

holiday

,City,Country,Latitude,Longitude,Date,Temperature,Humidity,Cloudiness,Wind Speed
0,chuy,UY,-33.6971,-53.4616,2021-01-14 12:05:56,22.03,51.0,0.0,4.64
1,saldanha,ZA,-33.0117,17.9442,2021-01-14 12:06:09,28.00,47.0,0.0,5.14
2,sur,OM,22.5667,59.5289,2021-01-14 12:06:16,23.10,52.0,0.0,5.76
3,rocha,UY,-34.4833,-54.3333,2021-01-14 12:06:37,22.57,42.0,0.0,3.72
4,mildura,AU,-34.2000,142.1500,2021-01-14 12:06:51,20.80,32.0,0.0,5.14
5,oussouye,SN,12.4850,-16.5469,2021-01-14 12:07:04,27.98,36.0,0.0,5.14
6,najran,SA,17.4924,44.1277,2021-01-14 12:07:07,24.00,23.0,0.0,2.06
7,kingaroy,AU,-26.5333,151.8333,2021-01-14 12:07:09,23.00,73.0,0.0,3.09
8,mar del plata,AR,-38.0023,-57.5575,2021-01-14 12:04:35,20.31,56.0,0.0,5.14
9,tessalit,ML,20.1986,1.0114,2021-01-14 12:07:58,29.06,12.0,0.0,2.77


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [89]:
column_names = ["Hotel Name"]

hotel_df = pd.DataFrame(columns = column_names)
hotel_df

,Hotel Name


In [134]:
#Test adding values to dataframe
hotel_df.loc[0] = 10.    #['name' + str(i)] + list(randint(10, size=2))
hotel_df

,Hotel Name
0,10.0


* Set parameters to search for hotels with 5000 meters.

In [97]:
#TEST RESPONSE and json check
# geocoordinates

target_coordinates = "43.6187102, -116.2146068"
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# convert response to json
places_data = response.json()

# Print the json (pretty printed)
print(json.dumps(places_data, indent=4, sort_keys=True))

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 43.6139805,
                    "lng": -116.2037792
                },
                "viewport": {
                    "northeast": {
                        "lat": 43.61524747989272,
                        "lng": -116.2022788701073
                    },
                    "southwest": {
                        "lat": 43.61254782010728,
                        "lng": -116.2049785298927
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "The Grove Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 553,
                    "html_attributions": [
                        "<a 

In [93]:
# geocoordinates
#target_coordinates = "43.6187102, -116.2146068"
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {
    #"location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
#response = requests.get(base_url, params=params)


* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap

In [139]:
#WORKING ON THIS ONE
# Loop through the list of coordinates and perform a request for data on each
for c in range(0, len(holiday)):
#for c in range(0, 10):
    
    lat = holiday["Latitude"][c]
    long = holiday["Longitude"][c]
    
    #print(lat, long)
    
    params = {
    "location": f"{lat}, {long}",
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}
    
    #print(params)
    
    response = requests.get(base_url, params=params)
    #print(response)
    
    # convert response to json
    try:
        #print("Processing Record " + str((holiday.index[c])+1) + " | " +  holiday["City"])
        places_data = response.json()
        #print(json.dumps(places_data, indent=4, sort_keys=True))
        #print(places_data["results"][0]["name"])
        
        hotelname = places_data["results"][0]["name"]
        print(hotelname)
        
        hotel_df.loc[c] = hotelname
        
        
        
        #hotel_df["Hotel Name"]['c'] = places_data["results"][0]["name"]
   
    except IndexError:
        print("No information")
    
    
   # Print the name and address of the first restaurant that appears
    #print(places_data["results"][0]["name"])
    #print(places_data["results"][0]["vicinity"])
    


Bertelli Chuí Hotel
Hoedjiesbaai Hotel
Sur Plaza Hotel
No information
Quality Hotel Mildura Grand
Campement ALULUM
No information
Ascot Lodge Motel Kingaroy
Hotel Las Rocas
No information
No information
Sheraton Oman Hotel
Pasyadahan Beach Resort
Hotel Park Center
Hostal Platinum Class
The Point Coolum Beach - Ocean View Accommodation
Hôtel Kasa Afrikana
Otana Hotel
Chelpasin
Bamboo Bungalow


In [140]:
hotel_df

,Hotel Name
0,Bertelli Chuí Hotel
1,Hoedjiesbaai Hotel
2,Sur Plaza Hotel
4,Quality Hotel Mildura Grand
5,Campement ALULUM
7,Ascot Lodge Motel Kingaroy
8,Hotel Las Rocas
11,Sheraton Oman Hotel
12,Pasyadahan Beach Resort
13,Hotel Park Center


In [141]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'City'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
